In [1]:
import pandas as pd
import numpy as np
import torch

In [6]:
data = pd.read_csv('data-stats-combs.csv', dtype=str)
data

,Do you want to be in society?,Do you make an attempt to socialise?,Do you consume things to alter your perspective? (Alcohol and stuff),Are you depressed?,Do you try to improve yourself?,Would you compromise with a friend?,If you and another person were both thirsty and you only had enough water for one person; would you share with the other person?,If a colleague invited you to do something with them; how would you answer? (Date; hang-out; party; etc.),Are you anxious of social interaction?,Do you think you fit in with other people?,...,What is your biological sex?,How tall are you?,How attractive are you (from 0 to 10),What is your BMI?,How many friends and colleagues do you currently have?,Rate your life right now.,Which climate do you live in?,X,Y,result
0,Y,Y,D,0,A,Y,N,N,0,N,...,male,11,21,31,41,51,tropical,0.0,-0.25,Cool normie
1,D,Y,A,N,N,A,Y,N,A,A,...,male,11,21,31,41,51,dry,-6.0,4.5,Failed normie
2,D,A,Y,Y,0,A,N,N,A,A,...,male,11,21,31,41,51,mild,0.0,4.25,Niche normie
3,A,D,D,D,A,0,N,Y,D,D,...,male,11,21,31,41,51,snow,-5.0,-2.25,Sperg
4,0,N,D,A,Y,Y,Y,Y,N,N,...,male,11,21,31,41,51,polar,-5.0,4.25,Failed normie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,D,D,N,D,Y,Y,N,Y,D,A,...,othersex,15,24,34,43,54,tropical,-2,0,Lame normie
14396,D,A,Y,Y,D,A,N,N,D,0,...,othersex,15,24,34,43,54,dry,1,-1.25,Cool normie
14397,Y,Y,N,D,N,0,N,Y,N,0,...,othersex,15,24,34,43,54,mild,-3,-0.25,Lame normie
14398,A,D,A,0,A,A,Y,N,Y,N,...,othersex,15,24,34,43,54,snow,-2,4.5,Niche normie


In [7]:
XY = data.values[:,-3:-1].astype(float)
XY

array([[ 0.  , -0.25],
       [-6.  ,  4.5 ],
       [ 0.  ,  4.25],
       ...,
       [-3.  , -0.25],
       [-2.  ,  4.5 ],
       [ 2.  , -3.25]])

In [8]:
# one-hot encode the categorical data
answers = pd.get_dummies(data[data.columns[:-3]])
answers

,Do you want to be in society?_0,Do you want to be in society?_A,Do you want to be in society?_D,Do you want to be in society?_N,Do you want to be in society?_Y,Do you make an attempt to socialise?_0,Do you make an attempt to socialise?_A,Do you make an attempt to socialise?_D,Do you make an attempt to socialise?_N,Do you make an attempt to socialise?_Y,...,How many friends and colleagues do you currently have?_43,Rate your life right now._51,Rate your life right now._52,Rate your life right now._53,Rate your life right now._54,Which climate do you live in?_dry,Which climate do you live in?_mild,Which climate do you live in?_polar,Which climate do you live in?_snow,Which climate do you live in?_tropical
0,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
14396,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,0
14397,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0
14398,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


In [9]:
answers = answers.values
answers = answers.astype(int)
XY = XY.astype(float)
dataset = torch.utils.data.DataLoader(list(zip(torch.from_numpy(answers), torch.from_numpy(XY))), batch_size=1)

In [12]:
all_weights = pd.read_csv('weights.csv', index_col=0)
x_weights = all_weights.loc['X']
y_weights = all_weights.loc['Y']
# see the diffs after rounding the weights
x_tensor = torch.from_numpy(x_weights.values).float()
y_tensor = torch.from_numpy(y_weights.values).float()
diffs = []
for ans, res in dataset:
    ans, res = ans.float(), res.float()
    pred_res = torch.from_numpy(np.array([[(ans@x_tensor.T).item(),(ans@y_tensor.T).item()]]))
    diff = res.detach().numpy().flatten()-pred_res.detach().numpy().flatten()
    diffs.append(diff)
diffs = np.array(diffs)
stat_corrections = pd.DataFrame(np.hstack((data[data.columns[-10:-3]], diffs)), 
                                columns=list(data.columns[-10:-3])+['difference_X', 'difference_Y'])
stat_corrections.to_csv('stat-corrections.csv', index=False)
stat_corrections

,What is your biological sex?,How tall are you?,How attractive are you (from 0 to 10),What is your BMI?,How many friends and colleagues do you currently have?,Rate your life right now.,Which climate do you live in?,difference_X,difference_Y
0,male,11,21,31,41,51,tropical,-2,-0.25
1,male,11,21,31,41,51,dry,-2,-0.25
2,male,11,21,31,41,51,mild,-2,-0.25
3,male,11,21,31,41,51,snow,-2,-0.25
4,male,11,21,31,41,51,polar,-2,-0.25
...,...,...,...,...,...,...,...,...,...
14395,othersex,15,24,34,43,54,tropical,-2,-0.25
14396,othersex,15,24,34,43,54,dry,-2,-0.25
14397,othersex,15,24,34,43,54,mild,-2,-0.25
14398,othersex,15,24,34,43,54,snow,-2,-0.25


In [16]:
def classify(x, y):
    if y > 6:
        if x < -6:
            return 'Social outcast'
        elif x < 0:
            return 'Simp'
        elif x < 6:
            return 'Party dude'
        else:
            return 'Chad'
    elif y > 0:
        if x < -4:
            return 'Failed normie'
        elif x < 0:
            return 'Niche normie'
        elif x < 5:
            return 'Normie'
        else:
            return 'Turbo normie'
    elif y > -6:
        if x < -4:
            return 'Sperg'
        elif x < 0:
            return 'Lame normie'
        elif x < 6:
            return 'Cool normie'
        else:
            return 'Based normie'
    else:
        if x < -6:
            return 'Wizard'
        elif x < 0:
            return 'Edge Lord'
        elif x < 5:
            return 'Sadboy'
        else:
            return 'Sociopath'
stats_columns = pd.get_dummies(data[data.columns[:-3]]).columns[-28:]
show_val = lambda encoding: encoding.split('_')[1]
for ans, res in dataset:
    ans, res = ans.float(), res.float()
    stats = ans[:,-28:-3]
    pred_res = torch.from_numpy(np.array([[(ans@x_tensor.T).item(),(ans@y_tensor.T).item()]]))
    stuff = list(map(show_val, stats_columns[np.nonzero(stats.numpy().flatten())].values))
    print(stuff)
# TODO: rename data-with-stats.csv to data-m111111.csv and remove the stats columns
# TODO: delete the old data-m111111-ignore-stats.csv

['male', '11', '21', '31', '41', '51']
['male', '11', '21', '31', '41', '51', 'dry']
['male', '11', '21', '31', '41', '51', 'mild']
['male', '11', '21', '31', '41', '51']
['male', '11', '21', '31', '41', '51']
['male', '11', '21', '31', '41', '52']
['male', '11', '21', '31', '41', '52', 'dry']
['male', '11', '21', '31', '41', '52', 'mild']
['male', '11', '21', '31', '41', '52']
['male', '11', '21', '31', '41', '52']
['male', '11', '21', '31', '41', '53']
['male', '11', '21', '31', '41', '53', 'dry']
['male', '11', '21', '31', '41', '53', 'mild']
['male', '11', '21', '31', '41', '53']
['male', '11', '21', '31', '41', '53']
['male', '11', '21', '31', '41', '54']
['male', '11', '21', '31', '41', '54', 'dry']
['male', '11', '21', '31', '41', '54', 'mild']
['male', '11', '21', '31', '41', '54']
['male', '11', '21', '31', '41', '54']
['male', '11', '21', '31', '42', '51']
['male', '11', '21', '31', '42', '51', 'dry']
['male', '11', '21', '31', '42', '51', 'mild']
['male', '11', '21', '31', '

['male', '11', '21', '34', '42', '51']
['male', '11', '21', '34', '42', '51']
['male', '11', '21', '34', '42', '52']
['male', '11', '21', '34', '42', '52', 'dry']
['male', '11', '21', '34', '42', '52', 'mild']
['male', '11', '21', '34', '42', '52']
['male', '11', '21', '34', '42', '52']
['male', '11', '21', '34', '42', '53']
['male', '11', '21', '34', '42', '53', 'dry']
['male', '11', '21', '34', '42', '53', 'mild']
['male', '11', '21', '34', '42', '53']
['male', '11', '21', '34', '42', '53']
['male', '11', '21', '34', '42', '54']
['male', '11', '21', '34', '42', '54', 'dry']
['male', '11', '21', '34', '42', '54', 'mild']
['male', '11', '21', '34', '42', '54']
['male', '11', '21', '34', '42', '54']
['male', '11', '21', '34', '43', '51']
['male', '11', '21', '34', '43', '51', 'dry']
['male', '11', '21', '34', '43', '51', 'mild']
['male', '11', '21', '34', '43', '51']
['male', '11', '21', '34', '43', '51']
['male', '11', '21', '34', '43', '52']
['male', '11', '21', '34', '43', '52', 'dry

['male', '11', '22', '33', '43', '53', 'mild']
['male', '11', '22', '33', '43', '53']
['male', '11', '22', '33', '43', '53']
['male', '11', '22', '33', '43', '54']
['male', '11', '22', '33', '43', '54', 'dry']
['male', '11', '22', '33', '43', '54', 'mild']
['male', '11', '22', '33', '43', '54']
['male', '11', '22', '33', '43', '54']
['male', '11', '22', '34', '41', '51']
['male', '11', '22', '34', '41', '51', 'dry']
['male', '11', '22', '34', '41', '51', 'mild']
['male', '11', '22', '34', '41', '51']
['male', '11', '22', '34', '41', '51']
['male', '11', '22', '34', '41', '52']
['male', '11', '22', '34', '41', '52', 'dry']
['male', '11', '22', '34', '41', '52', 'mild']
['male', '11', '22', '34', '41', '52']
['male', '11', '22', '34', '41', '52']
['male', '11', '22', '34', '41', '53']
['male', '11', '22', '34', '41', '53', 'dry']
['male', '11', '22', '34', '41', '53', 'mild']
['male', '11', '22', '34', '41', '53']
['male', '11', '22', '34', '41', '53']
['male', '11', '22', '34', '41', '5

['male', '11', '23', '33', '41', '54', 'mild']
['male', '11', '23', '33', '41', '54']
['male', '11', '23', '33', '41', '54']
['male', '11', '23', '33', '42', '51']
['male', '11', '23', '33', '42', '51', 'dry']
['male', '11', '23', '33', '42', '51', 'mild']
['male', '11', '23', '33', '42', '51']
['male', '11', '23', '33', '42', '51']
['male', '11', '23', '33', '42', '52']
['male', '11', '23', '33', '42', '52', 'dry']
['male', '11', '23', '33', '42', '52', 'mild']
['male', '11', '23', '33', '42', '52']
['male', '11', '23', '33', '42', '52']
['male', '11', '23', '33', '42', '53']
['male', '11', '23', '33', '42', '53', 'dry']
['male', '11', '23', '33', '42', '53', 'mild']
['male', '11', '23', '33', '42', '53']
['male', '11', '23', '33', '42', '53']
['male', '11', '23', '33', '42', '54']
['male', '11', '23', '33', '42', '54', 'dry']
['male', '11', '23', '33', '42', '54', 'mild']
['male', '11', '23', '33', '42', '54']
['male', '11', '23', '33', '42', '54']
['male', '11', '23', '33', '43', '5

['male', '12', '21', '31', '43', '51']
['male', '12', '21', '31', '43', '52']
['male', '12', '21', '31', '43', '52', 'dry']
['male', '12', '21', '31', '43', '52', 'mild']
['male', '12', '21', '31', '43', '52']
['male', '12', '21', '31', '43', '52']
['male', '12', '21', '31', '43', '53']
['male', '12', '21', '31', '43', '53', 'dry']
['male', '12', '21', '31', '43', '53', 'mild']
['male', '12', '21', '31', '43', '53']
['male', '12', '21', '31', '43', '53']
['male', '12', '21', '31', '43', '54']
['male', '12', '21', '31', '43', '54', 'dry']
['male', '12', '21', '31', '43', '54', 'mild']
['male', '12', '21', '31', '43', '54']
['male', '12', '21', '31', '43', '54']
['male', '12', '21', '32', '41', '51']
['male', '12', '21', '32', '41', '51', 'dry']
['male', '12', '21', '32', '41', '51', 'mild']
['male', '12', '21', '32', '41', '51']
['male', '12', '21', '32', '41', '51']
['male', '12', '21', '32', '41', '52']
['male', '12', '21', '32', '41', '52', 'dry']
['male', '12', '21', '32', '41', '52

...

['male', '13', '22', '31', '43', '52', 'mild']
['male', '13', '22', '31', '43', '52']
['male', '13', '22', '31', '43', '52']
['male', '13', '22', '31', '43', '53']
['male', '13', '22', '31', '43', '53', 'dry']
['male', '13', '22', '31', '43', '53', 'mild']
['male', '13', '22', '31', '43', '53']
['male', '13', '22', '31', '43', '53']
['male', '13', '22', '31', '43', '54']
['male', '13', '22', '31', '43', '54', 'dry']
['male', '13', '22', '31', '43', '54', 'mild']
['male', '13', '22', '31', '43', '54']
['male', '13', '22', '31', '43', '54']
['male', '13', '22', '32', '41', '51']
['male', '13', '22', '32', '41', '51', 'dry']
['male', '13', '22', '32', '41', '51', 'mild']
['male', '13', '22', '32', '41', '51']
['male', '13', '22', '32', '41', '51']
['male', '13', '22', '32', '41', '52']
['male', '13', '22', '32', '41', '52', 'dry']
['male', '13', '22', '32', '41', '52', 'mild']
['male', '13', '22', '32', '41', '52']
['male', '13', '22', '32', '41', '52']
['male', '13', '22', '32', '41', '5

['male', '13', '23', '31', '41', '53']
['male', '13', '23', '31', '41', '53', 'dry']
['male', '13', '23', '31', '41', '53', 'mild']
['male', '13', '23', '31', '41', '53']
['male', '13', '23', '31', '41', '53']
['male', '13', '23', '31', '41', '54']
['male', '13', '23', '31', '41', '54', 'dry']
['male', '13', '23', '31', '41', '54', 'mild']
['male', '13', '23', '31', '41', '54']
['male', '13', '23', '31', '41', '54']
['male', '13', '23', '31', '42', '51']
['male', '13', '23', '31', '42', '51', 'dry']
['male', '13', '23', '31', '42', '51', 'mild']
['male', '13', '23', '31', '42', '51']
['male', '13', '23', '31', '42', '51']
['male', '13', '23', '31', '42', '52']
['male', '13', '23', '31', '42', '52', 'dry']
['male', '13', '23', '31', '42', '52', 'mild']
['male', '13', '23', '31', '42', '52']
['male', '13', '23', '31', '42', '52']
['male', '13', '23', '31', '42', '53']
['male', '13', '23', '31', '42', '53', 'dry']
['male', '13', '23', '31', '42', '53', 'mild']
['male', '13', '23', '31', '

['male', '13', '23', '34', '42', '53']
['male', '13', '23', '34', '42', '53', 'dry']
['male', '13', '23', '34', '42', '53', 'mild']
['male', '13', '23', '34', '42', '53']
['male', '13', '23', '34', '42', '53']
['male', '13', '23', '34', '42', '54']
['male', '13', '23', '34', '42', '54', 'dry']
['male', '13', '23', '34', '42', '54', 'mild']
['male', '13', '23', '34', '42', '54']
['male', '13', '23', '34', '42', '54']
['male', '13', '23', '34', '43', '51']
['male', '13', '23', '34', '43', '51', 'dry']
['male', '13', '23', '34', '43', '51', 'mild']
['male', '13', '23', '34', '43', '51']
['male', '13', '23', '34', '43', '51']
['male', '13', '23', '34', '43', '52']
['male', '13', '23', '34', '43', '52', 'dry']
['male', '13', '23', '34', '43', '52', 'mild']
['male', '13', '23', '34', '43', '52']
['male', '13', '23', '34', '43', '52']
['male', '13', '23', '34', '43', '53']
['male', '13', '23', '34', '43', '53', 'dry']
['male', '13', '23', '34', '43', '53', 'mild']
['male', '13', '23', '34', '

['male', '13', '24', '34', '41', '51', 'dry']
['male', '13', '24', '34', '41', '51', 'mild']
['male', '13', '24', '34', '41', '51']
['male', '13', '24', '34', '41', '51']
['male', '13', '24', '34', '41', '52']
['male', '13', '24', '34', '41', '52', 'dry']
['male', '13', '24', '34', '41', '52', 'mild']
['male', '13', '24', '34', '41', '52']
['male', '13', '24', '34', '41', '52']
['male', '13', '24', '34', '41', '53']
['male', '13', '24', '34', '41', '53', 'dry']
['male', '13', '24', '34', '41', '53', 'mild']
['male', '13', '24', '34', '41', '53']
['male', '13', '24', '34', '41', '53']
['male', '13', '24', '34', '41', '54']
['male', '13', '24', '34', '41', '54', 'dry']
['male', '13', '24', '34', '41', '54', 'mild']
['male', '13', '24', '34', '41', '54']
['male', '13', '24', '34', '41', '54']
['male', '13', '24', '34', '42', '51']
['male', '13', '24', '34', '42', '51', 'dry']
['male', '13', '24', '34', '42', '51', 'mild']
['male', '13', '24', '34', '42', '51']
['male', '13', '24', '34', '

KeyboardInterrupt: 